In [23]:
# Data Handling
import pandas as pd
import numpy as np
import json


# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from IPython.display import Markdown as md

# Data import
import requests
from io import StringIO, BytesIO
from datetime import datetime
pd.set_option('display.max_columns', None)

# Leaflet maps via folium
import folium
import folium.plugins
from folium.plugins import HeatMap, TimestampedGeoJson
from branca.element import Template, MacroElement

# Surpress IPython warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Maximum number of sensor location that openAQ API retrieves
LOCATION_NUMBER_LIMIT = 100000

# WGS84 latitude, longitude pair of London, UK
LONDON_UK_COORDINATES = [51.5074, -0.1278] 

# Scoping radius in meters
RADIUS = 20000 

# URL for request
BASE_URL = 'https://api.openaq.org/v2/locations'

<img src="./data/London20kmRadius.png" style="float: center;" alt="airbnb logo" width="500"/>

<br>
<div style="text-align: caption;">
    Figure 1: Map of London, UK (20 km radius). 
    Retrieved from: <a href="https://radiusmap.traveltime.com/?lat=51.509865&lng=-0.118092&dist=km&radius=20&place=London">here</a>
   <br>
    <br>
</div>

In [3]:
def get_date_time(prompt):
    """
    Requests and validates user's input of date and time in ISO 8601 format with timezone offset.
    
    This function will keep prompting the user for input until a valid date and time string is entered.
    
    Args:
        prompt (str): The prompt that will be shown to the user when asking for input.

    Returns:
        datetime.datetime: The validated date and time entered by the user.

    Examples:
    To test this function, you will have to mock user input because it uses the input() function. 
    This isn't directly supported by doctest, so it isn't included here. In a testing suite like pytest, 
    you could use the pytest-mock or unittest.mock library to simulate user input.
    
    >>> from datetime import datetime
    >>> isinstance(get_date_time("Please enter a date and time: "), datetime)
    True
    """
    while True:
        date_str = input(prompt)
        try:
            valid_date = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S%z")
            print("The date and time you entered is valid.")
            break
        except ValueError:
            print("Invalid date and time. Please try again.")
    return valid_date

def get_date_range():
    """
    Requests and validates user's input of a date range in ISO 8601 format with timezone offset.
    
    This function will keep prompting the user for input until valid date and time strings are entered.
    
    Returns:
        tuple: The validated date range entered by the user.

    Examples:
    Testing this function with doctest is tricky, because it requires user input.
    """

    date_from = get_date_time("Please enter the start date and time in ISO 8601 format with timezone offset (YYYY-MM-DDTHH:MM:SS+HH:MM): ")
    date_to = get_date_time("Please enter the end date and time in ISO 8601 format with timezone offset (YYYY-MM-DDTHH:MM:SS+HH:MM): ")
    return [date_from, date_to]


In [4]:
def ask_which_pollutant():
    """
    Asks the user to input the type of pollutant they want to investigate.

    The function will continue to prompt the user until a valid input ('pm25' or 'pm10') is given,
    or the user decides to quit.

    Returns:
        str or bool: Returns the pollutant as a string if a valid input was given, 
        otherwise False if the user chose to quit.

    Examples:
    To test this function manually, you can call it and observe its behavior with different inputs.

    >>> ask_which_pollutant()
    Enter the pollutant you want to investigate [pm25/pm10]: pm25
    'pm25'

    >>> ask_which_pollutant()
    Enter the pollutant you want to investigate [pm25/pm10]: test
    Invalid input. Quit? [y/n]: y
    False

    Note:
    Testing this function with doctest or a similar tool is tricky, because it requires user input. You can 
    manually test the function by calling it in your code and entering inputs when prompted. For automated 
    testing, you would need to use a testing framework like pytest or unittest that can handle mocking user input.
    """
    while True:
        pollutant = input("Enter the pollutant you want to investigate [pm25/pm10]: ")
        if pollutant in ['pm25', 'pm10']:
            return pollutant
        else:
            while True:
                quit = input("Invalid input. Quit? [y/n]: ")
                if quit.lower() == 'y':
                    return False
                elif quit.lower() == 'n':
                    break
                else:
                    print("Invalid response. Please enter 'y' or 'n'.")


In [5]:
def check_pollutant(df, x):
    """
    (Docstring omitted for brevity)
    """
    return x in df['parameter'].values


def fetch_and_check(location_id, location_name, date_from, date_to, limit, x):
    """
    Fetches air quality data from the OpenAQ API and checks if a given pollutant is present.

    Args:
        location_id (str): The ID of the location to fetch data for.
        date_from (str): The start of the date range to fetch data for, in ISO 8601 format.
        date_to (str): The end of the date range to fetch data for, in ISO 8601 format.
        limit (int): The maximum number of results to fetch.
        x (str): The pollutant to check for.

    Returns:
        bool: True if the pollutant is present in the fetched data, False otherwise.
    """
    params = {
        'location_id': location_id,
        'date_from': date_from,
        'date_to': date_to,
        'limit': limit
    }
    base_url = 'https://api.openaq.org/v2/measurements'
    response = requests.get(base_url, params=params)
    our_location_data = response.json()

    # Check if there are any measurements in the response
    if 'results' not in our_location_data:
        print(f"No measurements found for {location_name} with location ID: {location_id}")
        return False

    measurements = [{"date": row['date']['local'], "value":row['value'], "parameter":row["parameter"], "unit":row["unit"]} for row in our_location_data['results']]
    if len(measurements) > 0:
        df = pd.DataFrame.from_dict(measurements)
        return check_pollutant(df, x)
    else:
        return False



In [6]:
def execute(date_from, date_to):
    """
    Fetches air quality data for a given location and checks for a specific pollutant.
    
    This function fetches air quality data from the OpenAQ API for a given location within a certain 
    radius. It then checks whether a specific pollutant, chosen by the user, is present in the data.

    Returns:
        list: A list of location IDs where the specified pollutant is present.
        
    Example:
    Due to the nature of this function (API calls, user input), it's not feasible to provide a doctest.
    """
    params = {
        'coordinates': ','.join(map(str, LONDON_UK_COORDINATES)),
        'radius': RADIUS,
    }
    response = requests.get(BASE_URL, params=params)

    # Check if the request was successful
    if response.status_code != 200:
        raise Exception(f"Request failed with status {response.status_code}")

    data = response.json()

    # Check what keys exist in the data
    print("Data keys:", data.keys())
    
    # Try to access the 'results' key
    try:
        locations = pd.DataFrame(data['results'], columns=['city', 'entity', 'name', 'id'])
    except KeyError:
        print("The key 'results' does not exist in the data.")
        return

    location_id_lst = []
    pollutant = ask_which_pollutant()
    for i in range(len(locations['id'])):
        if fetch_and_check(locations['id'][i], locations['name'][i], date_from, date_to, LOCATION_NUMBER_LIMIT, pollutant):
            location_id_lst.append([locations['name'][i], locations['id'][i]])

    return location_id_lst

### Prompt user input to obtain the set of location id with respect to the specific pollutant


In [7]:
date_from, date_to = get_date_range()
# Location id in London which has the targetted pollutant 
target_location_id = execute(date_from, date_to)


Please enter the start date and time in ISO 8601 format with timezone offset (YYYY-MM-DDTHH:MM:SS+HH:MM): 2023-05-31T00:00:00+00:00
The date and time you entered is valid.
Please enter the end date and time in ISO 8601 format with timezone offset (YYYY-MM-DDTHH:MM:SS+HH:MM): 2023-06-02T00:00:00+00:00
The date and time you entered is valid.
Data keys: dict_keys(['meta', 'results'])
Enter the pollutant you want to investigate [pm25/pm10]: pm25


In [8]:
print(target_location_id)

[['Acton', 65995], ['Newham, Upton Park', 1010097], ['Fulham Riverside', 65568], ['LionFreta-BXH', 613094], ['London Borough of Newham, Upton Park', 73049], ['London Bloomsbury', 148], ['Camden Kerbside', 152], ['London Teddington Bushy Park', 158], ['London Westminster', 159], ['London Eltham', 147], ['London Marylebone Road', 154], ['London N. Kensington', 155], ['Southwark A2 Old Kent Road', 146], ['City of London - Farringdon Street', 225743], ['Greenwich - Plumstead High Street', 225751], ['Greenwich - John Harrison Way', 225755], ['Greenwich - Woolwich Flyover', 225761], ['Kingston Upon Thames - Tolworth Broadway', 225764], ['Lewisham - Deptford', 225766], ['Lambeth - Brixton Road', 225767], ['Newham - Cam Road', 225770], ['Newham - Wren Close', 225772], ['Southwark - Tower Bridge Road', 225787], ['Bexley - Belvedere West', 225705], ['Bexley - Belvedere', 225714], ['Brent - Ikea', 225723], ['Bromley - Harwood Avenue', 225725], ['Southwark - Elephant and Castle', 225793], ['Southw

## Now we want to produce a heat map representing the PM emission data 

First, we have to find the coordinates `latitude` and `longitude` of each location

In [9]:
location_ids = target_location_id

# Initialize an empty DataFrame to store all results
all_locations = pd.DataFrame(columns=['name', 'entity', 'latitude', 'longitude', 'id'])

for location_name, location_id in location_ids:
    url = f'https://api.openaq.org/v2/measurements?location_id={location_id}'
    response = requests.get(url)
    our_location = response.json()

    if 'results' not in our_location:
        print(f"No results for location_id {location_id}. Skipping.")
        continue

    locations = pd.DataFrame.from_dict({
        "name": row['location'],
        "entity": row['entity'],
        "coordinates": row['coordinates'],
        'id': row['locationId']
    } for row in our_location['results'])

    # extract latitude and longitude from 'coordinates' into their own columns
    locations['latitude'] = locations['coordinates'].apply(lambda d: d.get('latitude'))
    locations['longitude'] = locations['coordinates'].apply(lambda d: d.get('longitude'))

    # drop the 'coordinates' column as it's no longer needed
    locations = locations.drop(columns=['coordinates'])

    # append this location's data to the master DataFrame
    all_locations = all_locations.append(locations, ignore_index=True)

# drop duplicates from the master DataFrame and reset index
all_locations = all_locations.drop_duplicates(ignore_index=True)



No results for location_id 65995. Skipping.
No results for location_id 148. Skipping.
No results for location_id 154. Skipping.
No results for location_id 155. Skipping.


In [10]:
all_locations

,name,entity,latitude,longitude,id
0,"Newham, Upton Park",Person,51.534695,0.027728,1010097
1,Fulham Riverside,Community Organization,51.466972,-0.187184,65568
2,LionFreta-BXH,Community Organization,51.453888,0.136877,613094
3,"London Borough of Newham, Upton Park",Community Organization,51.534668,0.027725,73049
4,Camden Kerbside,Governmental Organization,51.544210,-0.175269,152
5,London Teddington Bushy Park,Governmental Organization,51.425286,-0.345606,158
6,London Westminster,Governmental Organization,51.494670,-0.131931,159
7,London Eltham,Governmental Organization,51.452580,0.070766,147
8,Southwark A2 Old Kent Road,Governmental Organization,51.480499,-0.059550,146
9,City of London - Farringdon Street,Governmental Organization,51.514525,-0.104516,225743


Then we have to include the PM emission levels data

In [11]:
def fetch_measurements(location_id, date_from, date_to, limit=5000):
    """Fetches air quality measurements from the OpenAQ API.

    Args:
        location_id (str): The id of the location to get measurements for.
        date_from (str): The start date in ISO 8601 format.
        date_to (str): The end date in ISO 8601 format.
        limit (int, optional): The maximum number of measurements to fetch. Defaults to 5000.

    Returns:
        DataFrame: The measurements for the given location and date range.
    """
    base_url = 'https://api.openaq.org/v2/measurements'
    params = {
        'location_id': location_id, 
        'date_from': date_from,
        'date_to': date_to,
        'limit': limit
    }

    response = requests.get(base_url, params=params)
    response.raise_for_status()

    data = response.json()['results']
    measurements = [{"date": row['date']['local'], "value":row['value'], "parameter":row["parameter"], "unit":row["unit"]} for row in data]

    df = pd.DataFrame(measurements)
    df['date'] = pd.to_datetime(df['date'])

    return df

# Initialize an empty DataFrame
df_all = pd.DataFrame()

# Loop over all target locations
for i in range(len(all_locations.index)):
    location_name = all_locations['name'][i]
    location_id = all_locations['id'][i]

    # Fetch measurements for the current location
    df = fetch_measurements(location_id, date_from, date_to, LOCATION_NUMBER_LIMIT)

    # Filter for 'parameter' == 'pm25'
    df = df[df['parameter'] == 'pm25']

    # Add the location name, id, latitude, and longitude to the DataFrame
    df['location_name'] = location_name
    df['location_id'] = location_id
    df['latitude'] = all_locations['latitude'][i]
    df['longitude'] = all_locations['longitude'][i]

    # Append the DataFrame to df_all
    df_all = df_all.append(df)

# Reset the index of df_all
df_all.reset_index(drop=True, inplace=True)



In [12]:
df_all

,date,value,parameter,unit,location_name,location_id,latitude,longitude
0,2023-05-31 02:00:00+01:00,0.452632,pm25,µg/m³,"Newham, Upton Park",1010097,51.534695,0.027728
1,2023-05-31 03:00:00+01:00,0.645833,pm25,µg/m³,"Newham, Upton Park",1010097,51.534695,0.027728
2,2023-05-31 04:00:00+01:00,0.720000,pm25,µg/m³,"Newham, Upton Park",1010097,51.534695,0.027728
3,2023-05-31 05:00:00+01:00,0.584211,pm25,µg/m³,"Newham, Upton Park",1010097,51.534695,0.027728
4,2023-05-31 06:00:00+01:00,0.916667,pm25,µg/m³,"Newham, Upton Park",1010097,51.534695,0.027728
...,...,...,...,...,...,...,...,...
1476,2023-06-01 03:00:00+01:00,6.900000,pm25,µg/m³,Tower Hamlets - Jubilee Park,290183,51.502988,-0.018022
1477,2023-06-01 04:00:00+01:00,7.100000,pm25,µg/m³,Tower Hamlets - Jubilee Park,290183,51.502988,-0.018022
1478,2023-06-01 05:00:00+01:00,7.100000,pm25,µg/m³,Tower Hamlets - Jubilee Park,290183,51.502988,-0.018022
1479,2023-06-01 06:00:00+01:00,6.100000,pm25,µg/m³,Tower Hamlets - Jubilee Park,290183,51.502988,-0.018022


In [13]:
print(df_all['value'].describe())

count    1481.000000
mean        6.753722
std         3.469041
min        -7.000000
25%         5.000000
50%         7.000000
75%         8.800000
max        30.500000
Name: value, dtype: float64


In [14]:
# Remove list that contains NaN values
df_all = df_all.dropna()
print(df_all['value'].describe())

count    1481.000000
mean        6.753722
std         3.469041
min        -7.000000
25%         5.000000
50%         7.000000
75%         8.800000
max        30.500000
Name: value, dtype: float64


In [15]:
# Remove all PM emission values that are negataive
df_all = df_all[df_all['value'] >= 0]
print(df_all['value'].describe())

count    1463.000000
mean        6.868805
std         3.322537
min         0.000000
25%         5.000000
50%         7.000000
75%         8.898333
max        30.500000
Name: value, dtype: float64


# Heat maps representing the PM emission data 

## Heat map of listing counts (PM10 emission)

In [16]:
df_counts = df_all.groupby('location_id').agg({
    'latitude': 'mean', 
    'longitude': 'mean',
    'location_id': 'size'
}).rename(columns={'location_id': 'counts'}).reset_index()



In [17]:
# Extract the earliest and latest dates
date_from = df_all['date'].min().strftime('%Y-%m-%d')
date_to = df_all['date'].max().strftime('%Y-%m-%d')

title_html = f'<h3 align="center" style="font-size:16px"><b>Listings count across London ({date_from} to {date_to})</b></h3>'

# Create a Map instance
m = folium.Map(location=LONDON_UK_COORDINATES, zoom_start=9.6)

# Create a HeatMap layer
heat_data = df_counts[['latitude', 'longitude', 'counts']].values
HeatMap(heat_data, radius=15).add_to(m)


folium.Choropleth(
    geo_data="./data/london-boroughs_1179.geojson",
    name="choropleth",
    data=df_counts,
    columns=["location_id", "counts"],
    key_on="feature.properties.id",
    fill_color='RdBu_r',
    fill_opacity=0.3,
    line_opacity=0.5,
    legend_name="Number of Data Points",
).add_to(m)

# Add a layer control panel to the map
folium.LayerControl().add_to(m)

m.get_root().html.add_child(folium.Element(title_html))

m.save('listingCounts.html')
# Display the map
display(m)

### Mean PM10 emission levels in London on average over time period chosen (Aggregated Heat Map)

In [18]:
# Aggregate your data to calculate the mean value for each location
df_mean = df_all.groupby('location_id')['value'].mean().reset_index(name='mean_value')


In [19]:
# Extract the earliest and latest dates
date_from = df_all['date'].min().strftime('%Y-%m-%d')
date_to = df_all['date'].max().strftime('%Y-%m-%d')

title_html = f'<h3 align="center" style="font-size:16px"><b>Mean PM10 emission levels in London ({date_from} to {date_to})</b></h3>'

# Create a Map instance
m = folium.Map(location=LONDON_UK_COORDINATES, zoom_start=9.6)

# Create a HeatMap layer
heat_data = df_all[['latitude', 'longitude', 'value']].values
HeatMap(heat_data, radius=15).add_to(m)

folium.Choropleth(
    geo_data="./data/london-boroughs_1179.geojson",
    name="choropleth",
    data=df_mean,
    columns=["location_id", "mean_value"],
    key_on="feature.properties.id",
    fill_color='RdBu_r',
    fill_opacity=0.3,
    line_opacity=0.5,
    legend_name="Mean PM10 Emission Levels (µg/m³)",
).add_to(m)

# Add a layer control panel to the map
folium.LayerControl().add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.save('MeanPM10.html')
# Display the map
display(m)



### Heat map represnting all PM10 emission levels 

In [36]:
from jinja2 import Template
from folium.features import MacroElement

# Extract the earliest and latest dates
date_from = df_all['date'].min().strftime('%Y-%m-%d')
date_to = df_all['date'].max().strftime('%Y-%m-%d')

# Create a Map instance
m = folium.Map(location=LONDON_UK_COORDINATES, zoom_start=9.6)

# Add a fixed title
title_html = """
<h3 align="center" style="font-size:16px"><b>PM10 emission levels in London ({date_from} to {date_to})</b></h3>
""".format(date_from=date_from, date_to=date_to)

m.get_root().html.add_child(folium.Element(title_html))

# Convert date into a format that's suitable for use with TimestampedGeoJson, in this case, timestamp
df_all['date'] = pd.to_datetime(df_all['date'], utc=True)
df_all['timestamp'] = df_all['date'].apply(lambda x: x.strftime('%Y-%m-%dT%H:%M:%S'))

# Create GeoJSON features
features = df_all.apply(
    lambda row: {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [row['longitude'], row['latitude']],
        },
        'properties': {
            'time': row['timestamp'],
            'style': {'color': 'red' if row['value'] >= 9.1
                      else '#E6A751' if row['value'] >= 8.0
                      else '#F2E3CD' if row['value'] >= 7.0
                      else '#E5EDF6' if row['value'] >= 5.9
                      else '#4C7DBC' if row['value'] >= 4.8
                      else '#4C56BC'},
            'icon': 'circle',
            'iconstyle': {
                'fillColor': 'red' if row['value'] >= 9.1
                else '#E6A751' if row['value'] >= 8.0
                else '#F2E3CD' if row['value'] >= 7.0
                else '#E5EDF6' if row['value'] >= 5.9
                else '#4C7DBC' if row['value'] >= 4.8
                else '#4C56BC',
                'fillOpacity': 0.8,
                'stroke': 'true',
                'radius': 5,
            },
            'popup': f"Location: {row['location_name']}<br>Value: {row['value']}"
        },
    },
    axis=1
).tolist()


# Define the template for the legend (HTML + CSS)
template = """
{% macro html(this, kwargs) %}
<div style="
    position: fixed; 
    bottom: 30px;
    right: 10px;
    width: 260px;
    height: 80px; 
    z-index:9999;
    font-size:14px;
    font-weight: bold;
    background-color: white;  # add a white background
    padding: 10px;  # add some padding
    ">
    <p><a style="color:#E6A751">&block;&nbsp;</a>High PM10 emission levels</p>
    <p><a style="color:#E5EDF6">&block;&nbsp;</a>Moderate PM10 emission levels</p>
    <p><a style="color:#4C56BC">&block;&nbsp;</a>Low PM10 emission levels</p>
</div>
{% endmacro %}
"""

# Create a HeatMap layer
heat_data = df_all[['latitude', 'longitude', 'value']].values
HeatMap(heat_data, radius=15).add_to(m)

# Create a Choropleth layer
folium.Choropleth(
    geo_data="./data/london-boroughs_1179.geojson",
    name="choropleth",
    data=df_mean,
    columns=["location_id", "mean_value"],
    key_on="feature.properties.id",
    fill_color='RdBu_r',
    fill_opacity=0.3,
    line_opacity=0.5,
    legend_name="Mean PM10 Emission Levels (µg/m³)",
).add_to(m)


# Add the time-stamped data to the map
TimestampedGeoJson(
    {'type': 'FeatureCollection', 'features': features},
    period='PT1H',
    add_last_point=True,
).add_to(m)

# Create a Tooltip for displaying the location name and value
tooltip = folium.features.Tooltip(text='', sticky=False)

# Add the tooltip to the map
tooltip.add_to(m)

# Add a layer control panel to the map
folium.LayerControl().add_to(m)

# Add the custom legend to the map
macro = MacroElement()
macro._template = Template(template)
m.get_root().add_child(macro)

m.save('AnimatedPM10.html')
# Display the map
display(m)
